In [2]:
from __future__ import print_function
import astropy
from glob import glob
from astropy.table import Table
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#sys.path.pop(0)
from astropy.io import fits
import shutil
import barycorrpy
barycorrpy.__version__ #This tutorial assumes v0.4.4
from barycorrpy import sample_script 
results = sample_script.run_sample() 
import neidspec

%matplotlib inline

Please join our Google Group to keep abreast of the latest versions and bug fixes - https://groups.google.com/g/barycorrpy
***********SUCCESS**************
All barycentric correction velocities,  and time stamp conversions match expected values.


# First, lets make sure that barycorrpy is working

# Import our package

In [3]:
lib = Table.read('/home/tehan/PycharmProjects/neidspec/neidspec/data/neidmasterfile/SpecMatchNEIDlt4200K.csv')
type(lib['TIC'].tolist()[0])

int

In [4]:
files = glob('/home/tehan/Documents/NeidSpecMatch/FITS/*.fits')
# hdul = fits.open('/home/tehan/Documents/NeidSpecMatch/library/neidL2_20240111T054653.fits')
hdul = fits.open(files[0])

In [5]:
N = neidspec.NEIDSpectrum('/home/tehan/Documents/NeidSpecMatch/library/neidL2_20240111T054653.fits',ccf_redshift=True)

FileNotFoundError: [Errno 2] No such file or directory: '/home/tehan/Documents/NeidSpecMatch/library/neidL2_20240111T054653.fits'

In [8]:
files = glob('/home/tehan/PycharmProjects/neidspecmatch/library/20240301_specmatch_nir/FITS/*.fits')
lib_files = []
tics = []
tic=0
hd=0
for i in range(len(files)):
    hdul = fits.open(files[i])
    # print(hdul[0].header['OBJECT'])
    # N = neidspec.NEIDSpectrum(files[i],ccf_redshift=True)
    if hdul[0].header['OBJECT'][:3] == 'TIC':
        if int(hdul[0].header['OBJECT'][4:]) in lib['TIC'].tolist():
            tic+=1
            print(hdul[0].header['OBJECT'])
            lib_files.append(files[i])
            tics.append(int(hdul[0].header['OBJECT'][4:]))
    elif hdul[0].header['OBJECT'][:2] == 'HD':
        if f"HD{hdul[0].header['OBJECT'][3:]}" in lib['source_name'].tolist():
            hd+=1
            print(hdul[0].header['OBJECT'])
            lib_files.append(files[i])
print(sorted(tics))
print(len(set(tics)))
print(tic,hd)
# for file_name in lib_files:
#     shutil.copy(file_name, '/home/tehan/Documents/NeidSpecMatch/FITS/')

TIC 384491142
TIC 24250831
TIC 278962913
TIC 369534824
TIC 325554331
TIC 115869504
TIC 236743275
TIC 53580461
TIC 277888583
TIC 252803603
TIC 294401924
TIC 105618383
TIC 318686860
TIC 307646256
TIC 328860504
TIC 230241767
TIC 455139555
TIC 239332587
TIC 138819293
TIC 450298091
TIC 436856183
TIC 36853511
TIC 404519959
TIC 413948621
TIC 409527090
TIC 268804174
TIC 365159477
HD 355784
TIC 207492082
[24250831, 36853511, 53580461, 105618383, 115869504, 138819293, 207492082, 230241767, 236743275, 239332587, 252803603, 268804174, 277888583, 278962913, 294401924, 307646256, 318686860, 325554331, 328860504, 365159477, 369534824, 384491142, 404519959, 409527090, 413948621, 436856183, 450298091, 455139555]
28
28 1


In [ ]:
have_spec=[]
for i in range(len(lib['TIC'].tolist())-2):
    if int(lib['TIC'].tolist()[i]) in tics:
        have_spec.append(1)
    else:
        have_spec.append(0)
print(have_spec)
lib = lib[:-2]
lib['have_spectra'] = have_spec

In [10]:
lib.write('SpecMatchNEIDlt4200K_.csv', format='csv')

OSError: File SpecMatchNEIDlt4200K_.csv already exists. If you mean to replace it then use the argument "overwrite=True".

In [11]:
libcsv = Table.read('/home/tehan/PycharmProjects/neidspecmatch/library/20240301_specmatch_nir/20210808_spec_mann_w_files_qual1_pass3.csv')
libcsv['qprog'] = libcsv['qprog'].astype('<U15')
libcsv['OBJECT_ID'] = libcsv['OBJECT_ID'].astype('<U30')

In [14]:
files = glob('/home/tehan/PycharmProjects/neidspecmatch/library/20240301_specmatch_nir/FITS/*.fits')
neidlibcsv=Table(names=libcsv.colnames, dtype=libcsv.dtype)
length = 0
print(len(files))
for i in range(len(files)):
    hdul = fits.open(files[i])
    for j in range(len(libcsv)):
        if hdul[0].header['OBJECT'].split(' ')[-1] in libcsv['simbadnames_x'].tolist()[j]:
            row = libcsv[j]
            row['OBJECT_ID'] = hdul[0].header['OBJECT']
            row['filename'] = f'.{files[i][41:]}'
            row['basenames'] = os.path.basename(files[i])
            row['exptime'] = hdul[0].header['EXPTIME']
            row['sn18'] = hdul[0].header['EXTSNR']
            row['qprog'] = hdul[0].header['PROGRAM']
            neidlibcsv.add_row(row)
    length += 1
    if len(neidlibcsv)<length:
        print(hdul[0].header['OBJECT'])
        print(files[i])
        length = len(neidlibcsv)
neidlibcsv.rename_column('sn18', 'EXTSNR')
neidlibcsv['qprog']
neidlibcsv.write('20240301_46stars.csv', format='csv', overwrite=True)

38


### This creates a 'Target' class instance within our object

- The config file for the object is by default saved to "../data/target_files/"
- If that file already exist, then it is read in from disk. If it does not exist, it queries Simbad and creates the file
- This way you can change the values within the file if it is already created

## Lets plot the spectrum 

In [ ]:
N.plot_order(102,deblazed=False)

In [ ]:
N.plot_order(102,deblazed=True)

In [ ]:
N.plot_order(101,deblazed=True)

In [ ]:
#We could also this the following way using the handy wavelength and flux attributes
o = 102
o -= 10

# Blazed
fig, ax = plt.subplots()
ax.plot(N.w_shifted[o],N.f[o])
ax.set_title("Blazed spectrum shifted to stellar restframe\n(corrected for barycentric and absolute RV motion)")

# Deblazed
fig, ax = plt.subplots()
ax.plot(N.w_shifted[o],N.f_debl[o])
ax.set_title("Deblazed spectrum shifted to stellar restframe\n(corrected for barycentric and absolute RV motion)")


In [ ]:
# Other attributes are:
print(N.w_shifted.shape) # shifted wavelength
print(N.w.shape)         # non-shifted original wavelength
print(N.f.shape)         # non-deblazed science flux (sky-subtracted)
print(N.f_debl.shape)    # deblazed science flux (sky-subtracted)
print(N.f_sci.shape)     # Science flux (not sky-subtracted)
print(N.e.shape)         # error for deblazed sky-subtracted science flux
print(N.object)          # Object
print(N.bjd)             # BJD_TDB midpoint
print(N.sn55)            # SNR for order 55
print(N.berv)            # barycentric correction in km/s

# CCFs

In [ ]:
# Lets calculate fast ccfs for well-behaved orders using the crosscorr package
v = np.linspace(-125.,125.,1501)
orders = [55,56,102]
ccf = N.calculate_ccf_for_orders(v,orders=orders,plot=True)

In [ ]:
fig, ax = plt.subplots()
o = 55
ax.plot(v,ccf[o - 10]/np.nanmedian(ccf[o - 10]))

In [ ]:
# Lets calculate fast ccfs for well-behaved orders using the crosscorr package
v = np.linspace(-125., 125., 1501)
orders = [55, 56, 102]
ccf = N.calculate_ccf_for_orders(v, orders=orders, plot=True)# Calculates on barycentric shifted (not abs RV shifted) and undeblazed version
# rv1 is just the argmin of the CCF, the rv2 is an actual Gaussian fit minimum to the CCF valley
# You can see that order 18 is pretty wonky -- likely tellurics
# orders = [55,56]
# rv1, rv2 = N.rvabs_for_orders(v,orders,plot=True,verbose=True)

In [ ]:
# Lets calculate fast ccfs for well-behaved orders using the crosscorr package
v = np.linspace(-125.,125.,1501)
orders = [55,56,57,94,95,]
ccf = N.calculate_ccf_for_orders(v,orders=orders,plot=True)

# Working with a collection of spectra

In [ ]:
import glob

In [ ]:
files = glob.glob("../neidspec/data/neid/spectra/*.fits")
HS = neidspec.NEIDSpecList(filelist=files)

In [ ]:
HS.df

In [ ]:
HS.sn55

In [ ]:
HS.objects

In [ ]:
N